In [ ]:
using Distributed
using Statistics
using SharedArrays

# Add 30 worker processes for parallel processing
addprocs(30)


# Definig the function to calculate the entanglement, quantum and classical discord
@everywhere begin
    using QuadGK
    using DiffEqBase
    using DiffEqCallbacks
    using LinearAlgebra
    using OrdinaryDiffEq
    using Statistics
    
    # Time evolution of correlation matrix along the mean-field equations
    function vectorfield_B(du, u, p, t)
        xA, pA, xB, pB = u[1:4]
        Sigma = reshape(u[5:end], 4, 4)
        Omega, Kappa, U, Mu, nth = p # Mu corresponds to the chemical potential in the 
                                     # Bose-Hubbard model will be kept zero throughout the code 
    
        Eb = xB^2 + pB^2
        Ea = xA^2 + pA^2
    
        A = [2 * U * xA * pA  U * (Ea + 2 * pA^2) - Mu  0  Omega / 2;
             -U * (Ea + 2 * xA^2) + Mu  -2 * U * xA * pA  -Omega / 2  0;
             0  Omega / 2  2 * U * xB * pB  U * (Eb + 2 * pB^2) - Mu;
             -Omega / 2  0  -U * (Eb + 2 * xB^2) + Mu  -2 * U * xB * pB]
    
        Q = (Kappa / 4) * [-Eb  0  -2 * xA * xB  -2 * xA * pB;
                            0  -Eb  -2 * pA * xB  -2 * pA * pB;
                            2 * xA * xB  2 * pA * xB  Ea  0;
                            2 * xA * pB  2 * pA * pB  0  Ea]
    
        Zp = (Kappa*(2*nth+1) / 4) * [Eb  0  pA * pB - xA * xB  -(xA * pB + xB * pA);
                             0  Eb  -(xA * pB + xB * pA)  xA * xB - pA * pB;
                             pA * pB - xA * xB  -(xA * pB + xB * pA)  Ea  0;
                             -(xA * pB + xB * pA)  xA * xB - pA * pB  0  Ea]
    
        du[1] = Omega * pB / 2 - Kappa * xA * Eb / 4 + U * pA * Ea - Mu * pA
        du[2] = -Omega * xB / 2 - Kappa * pA * Eb / 4 - U * xA * Ea + Mu * xA
        du[3] = Omega * pA / 2 + Kappa * xB * Ea / 4 + U * pB * Eb - Mu * pB
        du[4] = -Omega * xA / 2 + Kappa * pB * Ea / 4 - U * xB * Eb + Mu * xB
    
        du[5:end] = (A + Q) * Sigma + (Sigma * (A' + Q')) + Zp
    end
    
    # calculating entanglement using the correlation matrix
    function entanglement(sigma)
        sym = [0 1 0 0; -1 0 0 0; 0 0 0 1; 0 0 -1 0]
        lamb = [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1]
        sigma_pt = lamb * sigma * lamb
        eigs_pt = eigvals(2im * (sym * sigma_pt))
        eigs_pt_min = minimum(abs.(eigs_pt))
        ent = max(0, -log(eigs_pt_min))
        return ent
    end
    
    # calculating classical discord using the correlation matrix
    function discord_cl(sigma)
        sym = [0 1 0 0; -1 0 0 0; 0 0 0 1; 0 0 -1 0]
    
        c_alpha = det(2 * sigma[1:2, 1:2])
        c_beta = det(2 * sigma[3:4, 3:4])
        c_gamma = det(2 * sigma[1:2, 3:4])
        c_delta = det(2 * sigma)
    
        if (c_delta - c_alpha * c_beta)^2 <= (1 + c_beta) * c_gamma^2 * (c_alpha + c_delta)
            Emin = (2 * c_gamma^2 + (c_beta - 1) * (c_delta - c_alpha) + 2 * abs(c_gamma) * sqrt(abs(c_gamma^2 + (c_beta - 1) * (c_delta - c_alpha)))) / (c_beta - 1)^2
        else
            Emin = (c_alpha * c_beta - c_gamma^2 + c_delta - sqrt(abs(c_gamma^4 + (c_delta - c_alpha * c_beta)^2 - 2 * c_gamma^2 * (c_alpha * c_beta + c_delta)))) / (2 * c_beta)
        end
    
        disc = funx(sqrt(abs(c_alpha))) - funx(sqrt(abs(Emin)))
        return real(disc)
    end

    # calculating quantum discord using the correlation matrix    
    function discord_qm(sigma)
        sym = [0 1 0 0; -1 0 0 0; 0 0 0 1; 0 0 -1 0]
    
        c_alpha = det(2 * sigma[1:2, 1:2])
        c_beta = det(2 * sigma[3:4, 3:4])
        c_gamma = det(2 * sigma[1:2, 3:4])
        c_delta = det(2 * sigma)
    
        if (c_delta - c_alpha * c_beta)^2 <= (1 + c_beta) * c_gamma^2 * (c_alpha + c_delta)
            Emin = (2 * c_gamma^2 + (c_beta - 1) * (c_delta - c_alpha) + 2 * abs(c_gamma) * sqrt(abs(c_gamma^2 + (c_beta - 1) * (c_delta - c_alpha)))) / (c_beta - 1)^2
        else
            Emin = (c_alpha * c_beta - c_gamma^2 + c_delta - sqrt(abs(c_gamma^4 + (c_delta - c_alpha * c_beta)^2 - 2 * c_gamma^2 * (c_alpha * c_beta + c_delta)))) / (2 * c_beta)
        end
    
        eigs = eigvals(2im * (sym * sigma))
        # println("Eigenvalues: ", eigs)
        # Consider only real part if imaginary part is smaller than 1e-12
        eigs = [real(eigval) for eigval in eigs]
    
        
        v_m, v_p = sort(eigs)[3:4]
    
        disc = funx(sqrt(abs(c_beta))) - funx(v_p) - funx(v_m) + funx(sqrt(abs(Emin)))
        return real(disc)
    end
    
    
    
    # Checking positivity of the dynamcis
    function pos(sigma)
        sym = [0 1 0 0; -1 0 0 0; 0 0 0 1; 0 0 -1 0]
        # println(eigvals(sigma + 1im * sym / 2))
        return minimum(eigvals(sigma + 1im * sym / 2))
    end
        
    function fourier_transform(data, dt)
        n = length(data)
        fft_result = fft(data)
        fft_freq = fftfreq(n, dt)
        return fft_freq, fft_result
    end
    
    function funx(x)
        if x-1<0 && abs((x-1)/2)<0.01
            return (x+1)*log((x+1)/2)/2-(x-1)*log(abs((x-1)/2))/2
        else
            return (x+1)*log((x+1)/2)/2-(x-1)*log((x-1)/2)/2
        end
    end
end

# Data for Fig. 3(a) of main text and Fig. S2

In [ ]:
# Define your Omega_values and nth_values lists here

leng_arr=30

Omega_values = range(0, 1.5, length=leng_arr)
nth_values = range(0.00001, 3, length=leng_arr) #lower value of n_th is kept very low (10e-5) for the convergence of numerical results

# Initiating the arrays to store the correlations

ent_avg = SharedArray{Float64}(leng_arr, leng_arr)
qmD_avg = SharedArray{Float64}(leng_arr, leng_arr)
clD_avg = SharedArray{Float64}(leng_arr, leng_arr)


f_s = 10
end_timee = 4000
time_list = range(0, stop=end_timee, length=end_timee * f_s)

# Intial state for mean-field variables and correlation matrix
eps = 0.3
m0 = [sqrt(2 - eps^2), eps, sqrt(2 - eps^2), eps]
sigma0 = Diagonal([1, 1, 1, 1]) / 2
ini = [m0; vec(sigma0)]

kappa = 1
U = 0
Mu = 0 



@sync begin
    for (i, Omega) in enumerate(Omega_values)
        @distributed for j in 1:length(nth_values)
            p0 = [Omega, kappa, U, Mu, nth_values[j]]
            msol = ODEProblem(vectorfield_B, ini, (0, end_timee), p0)
            sol = solve(msol, DP8(), saveat=1/f_s, abstol=1e-18, reltol=1e-18)#, dt=1/f_s)
            ent_avg[i, j] = mean([entanglement(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)])
            qmD_avg[i, j] = mean(skipmissing([discord_qm(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)]))
            clD_avg[i, j] = mean(skipmissing([discord_cl(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)]))
            println(i, j)
        end
    end
end

# Remove the extra worker processes
rmprocs(workers())

using DelimitedFiles

open("ent_nth.dat", "w") do f
    writedlm(f, ent_avg)
end

open("qmD_nth.dat", "w") do f
    writedlm(f, qmD_avg)
end

open("clD_nth.dat", "w") do f
    writedlm(f, clD_avg)
end

# Data for Fig. 3(c) of main text and Fig. S3

In [ ]:
# Define your Omega_values and U_values lists here

leng_arr=60

Omega_values = range(0, 1.5, length=leng_arr)
U_values = range(-0.3, 0.3, length=leng_arr)

# Initiating the arrays to store the correlations

ent_avg = SharedArray{Float64}(leng_arr, leng_arr)
qmD_avg = SharedArray{Float64}(leng_arr, leng_arr)
clD_avg = SharedArray{Float64}(leng_arr, leng_arr)


f_s = 10
end_timee = 4000
time_list = range(0, stop=end_timee, length=end_timee * f_s)

# Intial state for mean-field variables and correlation matrix
eps = 0.3
m0 = [sqrt(2 - eps^2), eps, sqrt(2 - eps^2), eps]
sigma0 = Diagonal([1, 1, 1, 1]) / 2
ini = [m0; vec(sigma0)]

kappa = 1
nth = 0.00001 #the value of n_th is kept very low (10e-5) for the convergence of numerical results
Mu = 0 



@sync begin
    for (i, Omega) in enumerate(Omega_values)
        @distributed for j in 1:length(U_values)
            p0 = [Omega, kappa, U_values[j], Mu, nth]
            msol = ODEProblem(vectorfield_B, ini, (0, end_timee), p0)
            sol = solve(msol, DP8(), saveat=1/f_s, abstol=1e-18, reltol=1e-18)#, dt=1/f_s)
            ent_avg[i, j] = mean([entanglement(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)])
            qmD_avg[i, j] = mean(skipmissing([discord_qm(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)]))
            clD_avg[i, j] = mean(skipmissing([discord_cl(reshape(sol.u[i][5:end], 4, 4)) for i in 2:length(time_list)]))
            println(i, j)
        end
    end
end

# Remove the extra worker processes
rmprocs(workers())

using DelimitedFiles

open("ent_U.dat", "w") do f
    writedlm(f, ent_avg)
end

open("qmD_U.dat", "w") do f
    writedlm(f, qmD_avg)
end

open("clD_U.dat", "w") do f
    writedlm(f, clD_avg)
end